In [ ]:
import io
from PIL import Image
import os
import glob
from PyPDF2 import PdfReader
import fnmatch

import fnmatch
import zlib

def extract_images_from_pdf(pdf_file):
    images = []
    
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            try:
                xObject = page['/Resources']['/XObject'].get_object()
                if xObject is not None:
                    for obj in xObject:
                        # Check for Form XObjects
                        if xObject[obj]['/Type'] == '/XObject' and xObject[obj]['/Subtype'] == '/Form':
                            form_xobject = xObject[obj].get_object()
                            # Process image XObjects within the Form XObject
                            for image_xobject_name in form_xobject:
                                # Extract image data from PdfImageXObject
                                if isinstance(form_xobject[image_xobject_name], PdfReader.PdfImageXObject):
                                    image_data = form_xobject[image_xobject_name].get_data()
                                    image = Image.open(io.BytesIO(image_data))
                                    images.append(image)
            except KeyError:
                # Skip this page if '/Resources' or '/XObject' is not present
                print(f"Warning: Skipping page {page_num + 1} in {pdf_file} due to missing '/Resources' or '/XObject'.")
                continue
    
    return images


# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    
    # Extract images from the current PDF file
    extracted_images = extract_images_from_pdf(pdf_file)
    num_images = len(extracted_images)
    
    # Save the extracted images to files or perform further processing as needed.
    if num_images > 0:
        print(f"Found {num_images} image(s) in {pdf_file}. Saving images...")
        for i, image in enumerate(extracted_images):
            image.save(f'image_{i}.png', 'PNG')
        print(f"Images saved for {pdf_file}.")
    else:
        print(f"No images found in {pdf_file}.")

    print()  # Add an empty line for better readability between PDFs


In [ ]:
import PyPDF2
import io
from PIL import Image
import os
import glob

def extract_images_from_pdf(pdf_file):
    images = []
    
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            try:
                xObject = page['/Resources']['/XObject'].get_object()
                print(f"xObject: {xObject}")  # Debugging print statement
                if xObject is not None:
                    for obj in xObject:
                        print(f"obj: {obj}")  # Debugging print statement
                        if xObject[obj]['/Type'] == '/XObject' and xObject[obj]['/Subtype'] == '/Form':
                            form_xobject = xObject[obj].get_object()
                            print(f"form_xobject: {form_xobject}")  # Debugging print statement
                            
                            for image_xobject_name, image_xobject in form_xobject.items():
                                print(f"image_xobject_name: {image_xobject_name}, image_xobject: {image_xobject}")  # Debugging print statement
                                if isinstance(image_xobject, dict):
                                    if '/Subtype' not in image_xobject or image_xobject.get('/Subtype') == '/Image':
                                        image_obj = reader.get_object(image_xobject)
                                        image_data = image_obj.data
                                        image = Image.open(io.BytesIO(image_data))
                                        images.append(image)
            except KeyError:
                print(f"Warning: Skipping page {page_num + 1} in {pdf_file} due to missing '/Resources' or '/XObject'.")
                continue
            except Exception as e:  # Additional exception handling
                print(f"An exception occurred: {e}")
                continue
    
    return images

# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    
    # Extract images from the current PDF file
    extracted_images = extract_images_from_pdf(pdf_file)
    num_images = len(extracted_images)
    
    # Save the extracted images to files or perform further processing as needed.
    if num_images > 0:
        print(f"Found {num_images} image(s) in {pdf_file}. Saving images...")
        for i, image in enumerate(extracted_images):
            image.save(f'image_{i}.png', 'PNG')
        print(f"Images saved for {pdf_file}.")
    else:
        print(f"No images found in {pdf_file}.")

    print()  # Add an empty line for better readability between PDFs


In [ ]:
import os
import glob
import PyPDF2
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure
from pdfminer.pdfpage import PDFPage
from PIL import Image, ImageDraw
import io
from pdf2image import convert_from_path
from IPython.display import display
import numpy as np

def display_pdf_page_with_bounding_box(pdf_path, element, page_number=0):
    # Convert the PDF page to an image (PNG)
    dpi = 1000
    images = convert_from_path(pdf_path, dpi=dpi ,first_page=page_number + 1, last_page=page_number + 1)

    # Display the image with bounding box in the notebook
    if images:
        image = images[0]
        
        # Get the bounding box coordinates from element.bbox
        x0, y0, x1, y1 = element.bbox
        # the magic numbers
        padding = 5
        # correction PDF --> PIL
        startY = image.height - int(y0 * dpi/72) + padding
        endY   = image.height - int(y1   * dpi/72) - padding
        startX = int(x0 * dpi/72) + 1
        endX   = int(x1   * dpi/72) - 1
        startY, endY = endY, startY 
            
        # Crop the image to the bounding box
        cropped_image = image.crop((startX, startY, endX, endY))

        # Display the image with bounding box
        display(cropped_image)
        
        # turn image to array
        image_array = np.array(image)
        # get cropped box
        box = image_array[startY:endY,startX:endX,:]
        convert2pil_image = Image.fromarray(box)
        
        
        #show cropped box image
        # convert2pil_image.show()
        png = str(element.name) + ".pdf"
        convert2pil_image.save(png)

def save_figure_as_pdf(pdf_path, element, page_number=0):
    # Get the bounding box coordinates from element.bbox
    x0, y0, x1, y1 = element.bbox

    with open(pdf_path, 'rb') as file:
        reader = PdfFileReader(file)
        writer = PdfFileWriter()

        page = reader.getPage(page_number)
        page.cropBox.lowerLeft = (x0, y0)
        page.cropBox.upperRight = (x1, y1)
        writer.addPage(page)

        cropped_pdf_path = "cropped_page.pdf"
        with open(cropped_pdf_path, 'wb') as output_file:
            writer.write(output_file)

    print(f"Cropped PDF saved as {cropped_pdf_path}")

def extract_images_from_pdf(pdf_path):
    # Create the output folder if it does not exist


    resource_manager = PDFResourceManager()
    device = PDFPageAggregator(resource_manager, laparams=LAParams())
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            interpreter.process_page(page)
            layout = device.get_result()

            for element in layout:
                if isinstance(element, LTFigure):
                    # Specify the output path for the extracted figure
                    output_path = os.path.join(output_folder, f'figure_{page_number}.pdf')
                    # Example usage:
                    # Replace 'your_pdf_file.pdf' with the path to your PDF file
                    display_pdf_page_with_bounding_box(pdf_path, element, page_number)
                    
                    
                    save_figure_as_pdf(pdf_path, element, page_number)



def display_cropped_image(pdf_path):
    # Convert the cropped PDF region to an image using pdf2image
    images = pdf2image.convert_from_path(pdf_path)
    
    # Display the first image (assuming the cropped region is on the first page)
    if images:
        images[0].show()

# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    # Specify the output folder where you want to save the extracted figures
    output_folder = "output_folder"

    # Extract figures from the current PDF file
    extract_images_from_pdf(pdf_file, output_folder)

    print()  # Add an empty line for better readability between PDFs


In [ ]:
import os
import glob
import PyPDF2
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure
from pdfminer.pdfpage import PDFPage
from PIL import Image, ImageDraw
import io
from pdf2image import convert_from_path
from IPython.display import display
import numpy as np

def display_pdf_page_with_bounding_box(pdf_path, element, page_number=0):
    # Convert the PDF page to an image (PNG)
    images = convert_from_path(pdf_path, first_page=page_number + 1, last_page=page_number + 1)

    # Display the image with bounding box in the notebook
    if images:
        image = images[0]
        
        # Get the bounding box coordinates from element.bbox
        x0, y0, x1, y1 = element.bbox
        # the magic numbers
        dpi = 200/72
    
        # correction PDF --> PIL
        startY = 2200 - int(y0 * dpi) 
        endY   = 2200 - int(y1   * dpi)
        startX = int(x0 * dpi)  +1 
        endX   = int(x1   * dpi)
        startY, endY = endY, startY 

        # turn image to array
        image_array = np.array(image)
        # get cropped box
        box = image_array[startY:endY,startX:endX,:]
        convert2pil_image = Image.fromarray(box)
        
        
        #show cropped box image
        # convert2pil_image.show()
        png = "crop_" + str(element.name) + ".png"
        convert2pil_image.save(png)

    
def extract_images_from_pdf(pdf_path, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    resource_manager = PDFResourceManager()
    device = PDFPageAggregator(resource_manager, laparams=LAParams())
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            interpreter.process_page(page)
            layout = device.get_result()

            for element in layout:
                if isinstance(element, LTFigure):
                    # Specify the output path for the extracted figure
                    output_path = os.path.join(output_folder, f'figure_{page_number}.pdf')
                    # Example usage:
                    # Replace 'your_pdf_file.pdf' with the path to your PDF file
                    display_pdf_page_with_bounding_box(pdf_path, element, page_number=page_number)
                    # Save the figure as a standalone PDF
                    #save_figure_as_pdf(pdf_path, element, output_path)


def display_cropped_image(pdf_path):
    # Convert the cropped PDF region to an image using pdf2image
    images = pdf2image.convert_from_path(pdf_path)
    
    # Display the first image (assuming the cropped region is on the first page)
    if images:
        images[0].show()

# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    # Specify the output folder where you want to save the extracted figures
    output_folder = "output_folder"

    # Extract figures from the current PDF file
    extract_images_from_pdf(pdf_file, output_folder)

    print()  # Add an empty line for better readability between PDFs


In [5]:
import os
import glob
from PyPDF2 import PdfWriter, PdfReader
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure
from pdfminer.pdfpage import PDFPage
from PIL import Image, ImageDraw
import io
from pdf2image import convert_from_path
from IPython.display import display
import numpy as np

def display_pdf_page_with_bounding_box(pdf_path, element, page_number=0):
    # Convert the PDF page to an image (PNG)
    dpi = 300
    images = convert_from_path(pdf_path, dpi=dpi ,first_page=page_number + 1, last_page=page_number + 1)

    # Display the image with bounding box in the notebook
    if images:
        image = images[0]
        
        # Get the bounding box coordinates from element.bbox
        x0, y0, x1, y1 = element.bbox
        # the magic numbers

        # correction PDF --> PIL
        startY = image.height - int(y0 * dpi/72)
        endY   = image.height - int(y1   * dpi/72)
        startX = int(x0 * dpi/72)
        endX   = int(x1   * dpi/72)
        
        # Calculate the initial crop box coordinates
        startY = image.height - int(y0 * dpi/72)
        endY = image.height - int(y1 * dpi/72)
        startX = int(x0 * dpi/72)
        endX = int(x1 * dpi/72)

        # Initialize the padding
        padding = 5

        # Check if the padding exceeds the image boundaries
        if startY - padding < 0:
            startY = 0
        else:
            startY += padding

        if endY + padding > image.height:
            endY = image.height
        else:
            endY -= padding

        if startX - padding < 0:
            startX = 0
        else:
            startX -= padding

        if endX + padding > image.width:
            endX = image.width
        else:
            endX += padding
            
        startY, endY = endY, startY 
            
        # Crop the image to the bounding box
        cropped_image = image.crop((startX, startY, endX, endY))

        # Display the image with bounding box
        #display(cropped_image)
        
        # turn image to array
        image_array = np.array(image)
        # get cropped box
        box = image_array[startY:endY,startX:endX,:]
        convert2pil_image = Image.fromarray(box)
        
        # Create directory if it doesn't exist
        if not os.path.exists("Images"):
            os.makedirs("Images")
            
        #show cropped box image
        # convert2pil_image.show()
        png = "Images/" + str(element.name) + ".png"
        convert2pil_image.save(png)

def save_figure_as_pdf(pdf_path, element, page_number=0):
    # Get the bounding box coordinates of the figure
    # The dimensions of the page (assumed to be A4 in points)
    
    reader = PdfReader(pdf_path)
    writer = PdfWriter()
    page = reader.pages[page_number]
    
    
    page_height = page.mediabox[3]
    page_width = page.mediabox[2]

    # The bounding box coordinates
    x0, y0, x1, y1 = element.bbox

    padding = 1
    # Check if the padding exceeds the page boundaries
    if x0 - padding < 0:
        x0 = 0
    else:
        x0 -= padding

    if y0 - padding < 0:
        y0 = 0
    else:
        y0 -= padding

    if x1 + padding > page_width:
        x1 = page_width
    else:
        x1 += padding

    if y1 + padding > page_height:
        y1 = page_height
    else:
        y1 += padding

    (x0, y0, x1, y1) = (int(x0), int(y0), int(x1), int(y1))
    
    #y0 = page.mediabox[3] - int(y0)
    #y1 = page.mediabox[3] - int(y1)
    #y0, y1 = y1, y0 
        
    page.cropbox.lower_left = (x0, y1)
    page.cropbox.upper_right = (x1, y0)
    writer.add_page(page)
    # Create directory if it doesn't exist
    if not os.path.exists("Images"):
        os.makedirs("Images")
        
    # Save the new PDF with the selected figure
    output_path = "Images/" + str(element.name) + ".pdf"
    with open(output_path, 'wb') as output_file:
        writer.write(output_file)
        
def extract_images_from_pdf(pdf_path):

    resource_manager = PDFResourceManager()
    device = PDFPageAggregator(resource_manager, laparams=LAParams())
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            interpreter.process_page(page)
            layout = device.get_result()

            for element in layout:
                if isinstance(element, LTFigure):
                    # Specify the output path for the extracted figure
                    # Example usage:
                    # Replace 'your_pdf_file.pdf' with the path to your PDF file
                    display_pdf_page_with_bounding_box(pdf_path, element, page_number)
                    
                    save_figure_as_pdf(pdf_path, element, page_number)


# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    # Specify the output folder where you want to save the extracted figures

    # Extract figures from the current PDF file
    extract_images_from_pdf(pdf_file)

    print()  # Add an empty line for better readability between PDFs


Processing: c:\Users\mihao\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\PDF_Images\oAL_for_evolving_error_feedback_fuzzymodel_multiInno_Revision01.pdf



In [13]:
from pdfminer.layout import LAParams, LTTextBoxHorizontal
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
import os
import glob

def extract_text_from_pdf(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    resource_manager = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(resource_manager, laparams=laparams)
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            interpreter.process_page(page)
            layout = device.get_result()
            if page_number == 12:
                print('')
            for element in layout:
                if isinstance(element, LTTextBoxHorizontal):
                    with open(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_text.txt"), 'a') as text_file:
                        text_file.write(element.get_text())

current_directory = os.getcwd()
pdf_pattern = '*.pdf'
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))
output_folder = os.path.join(current_directory, 'Output')

for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    extract_text_from_pdf(pdf_file, output_folder)
    print()  # Add an empty line for better readability between PDFs


Processing: c:\Users\mihao\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\PDF_Images\oAL_for_evolving_error_feedback_fuzzymodel_multiInno_Revision01.pdf




In [ ]:
import os
import glob
from PyPDF2 import PdfWriter, PdfReader
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTFigure
from pdfminer.pdfpage import PDFPage
from PIL import Image, ImageDraw
import io
from pdf2image import convert_from_path
from IPython.display import display
import numpy as np

def display_pdf_page_with_bounding_box(pdf_path, element, page_number=0):
    # Convert the PDF page to an image (PNG)
    dpi = 300
    images = convert_from_path(pdf_path, dpi=dpi ,first_page=page_number + 1, last_page=page_number + 1)

    # Display the image with bounding box in the notebook
    if images:
        image = images[0]
        
        # Get the bounding box coordinates from element.bbox
        x0, y0, x1, y1 = element.bbox
        # the magic numbers

        # correction PDF --> PIL
        startY = image.height - int(y0 * dpi/72)
        endY   = image.height - int(y1   * dpi/72)
        startX = int(x0 * dpi/72)
        endX   = int(x1   * dpi/72)
        
        # Calculate the initial crop box coordinates
        startY = image.height - int(y0 * dpi/72)
        endY = image.height - int(y1 * dpi/72)
        startX = int(x0 * dpi/72)
        endX = int(x1 * dpi/72)

        # Initialize the padding
        padding = 5

        # Check if the padding exceeds the image boundaries
        if startY - padding < 0:
            startY = 0
        else:
            startY += padding

        if endY + padding > image.height:
            endY = image.height
        else:
            endY -= padding

        if startX - padding < 0:
            startX = 0
        else:
            startX -= padding

        if endX + padding > image.width:
            endX = image.width
        else:
            endX += padding
            
        startY, endY = endY, startY 
            
        # Crop the image to the bounding box
        cropped_image = image.crop((startX, startY, endX, endY))

        # Display the image with bounding box
        #display(cropped_image)
        
        # turn image to array
        image_array = np.array(image)
        # get cropped box
        box = image_array[startY:endY,startX:endX,:]
        convert2pil_image = Image.fromarray(box)
        
        # Create directory if it doesn't exist
        if not os.path.exists("Images"):
            os.makedirs("Images")
            
        #show cropped box image
        # convert2pil_image.show()
        png = "Images/" + str(element.name) + ".png"
        convert2pil_image.save(png)

def save_figure_as_pdf(pdf_path, element, page_number=0):
    # Get the bounding box coordinates of the figure
    # The dimensions of the page (assumed to be A4 in points)
    
    reader = PdfReader(pdf_path)
    writer = PdfWriter()
    page = reader.pages[page_number]
    
    
    page_height = page.mediabox[3]
    page_width = page.mediabox[2]

    # The bounding box coordinates
    x0, y0, x1, y1 = element.bbox

    padding = 1
    # Check if the padding exceeds the page boundaries
    if x0 - padding < 0:
        x0 = 0
    else:
        x0 -= padding

    if y0 - padding < 0:
        y0 = 0
    else:
        y0 -= padding

    if x1 + padding > page_width:
        x1 = page_width
    else:
        x1 += padding

    if y1 + padding > page_height:
        y1 = page_height
    else:
        y1 += padding

    (x0, y0, x1, y1) = (int(x0), int(y0), int(x1), int(y1))
    
    #y0 = page.mediabox[3] - int(y0)
    #y1 = page.mediabox[3] - int(y1)
    #y0, y1 = y1, y0 
        
    page.cropbox.lower_left = (x0, y1)
    page.cropbox.upper_right = (x1, y0)
    writer.add_page(page)
    # Create directory if it doesn't exist
    if not os.path.exists("Images"):
        os.makedirs("Images")
        
    # Save the new PDF with the selected figure
    output_path = "Images/" + str(element.name) + ".pdf"
    with open(output_path, 'wb') as output_file:
        writer.write(output_file)
        
def extract_images_from_pdf(pdf_path):

    resource_manager = PDFResourceManager()
    device = PDFPageAggregator(resource_manager, laparams=LAParams())
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            interpreter.process_page(page)
            layout = device.get_result()

            for element in layout:
                if isinstance(element, LTFigure):
                    # Specify the output path for the extracted figure
                    # Example usage:
                    # Replace 'your_pdf_file.pdf' with the path to your PDF file
                    display_pdf_page_with_bounding_box(pdf_path, element, page_number)
                    
                    save_figure_as_pdf(pdf_path, element, page_number)


# Get the current working directory
current_directory = os.getcwd()

# Pattern to match PDF files
pdf_pattern = '*.pdf'

# Use glob to find PDF files in the current directory
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))

# Process each PDF file
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    # Specify the output folder where you want to save the extracted figures

    # Extract figures from the current PDF file
    extract_images_from_pdf(pdf_file)

    print()  # Add an empty line for better readability between PDFs


Processing: c:\Users\mihao\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\PDF_Images\oAL_for_evolving_error_feedback_fuzzymodel_multiInno_Revision01.pdf



In [20]:
from pdfminer.layout import LAParams, LTTextBoxHorizontal
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
import os
import glob

def extract_text_from_pdf(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    resource_manager = PDFResourceManager()
    laparams = LAParams(line_overlap=1, char_margin=1.0, line_margin=1.0, word_margin=1, boxes_flow=-0.5)  # Modify as needed
    device = PDFPageAggregator(resource_manager, laparams=laparams)
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page_number, page in enumerate(PDFPage.get_pages(file)):
            try:
                interpreter.process_page(page)
                layout = device.get_result()

                for element in layout:
                    if isinstance(element, LTTextBoxHorizontal):
                        with open(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_text.txt"), 'a') as text_file:
                            text_file.write(element.get_text())
            except Exception as e:
                print(f"Error on page {page_number}: {e}")

current_directory = os.getcwd()
pdf_pattern = '*.pdf'
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))
output_folder = os.path.join(current_directory, 'Output')

for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    extract_text_from_pdf(pdf_file, output_folder)
    print()  # Add an empty line for better readability between PDFs


Processing: c:\Users\mihao\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\PDF_Images\oAL_for_evolving_error_feedback_fuzzymodel_multiInno_Revision01.pdf


KeyboardInterrupt: 

In [25]:
import fitz  # PyMuPDF
import os
import glob

def extract_text(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()

    with open(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_text.txt"), 'w') as f:
        f.write(text)

current_directory = os.getcwd()
pdf_pattern = '*.pdf'
pdf_files = glob.glob(os.path.join(current_directory, pdf_pattern))
output_folder = os.path.join(current_directory, 'Output')

for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    extract_text(pdf_file, output_folder)
    print()  # Add an empty line for better readability between PDFs

Processing: c:\Users\mihao\OneDrive - Univerza v Ljubljani\Doktorski_studij\Delo\PDF_Images\oAL_for_evolving_error_feedback_fuzzymodel_multiInno_Revision01.pdf



In [35]:
import urllib.request
import re
from scholarly import scholarly

def check_internet():
    try:
        urllib.request.urlopen('http://google.com', timeout=5)
        return True
    except urllib.request.URLError:
        return False

print(check_internet())

# Read the references.txt file
with open('references.txt', 'r') as f:
    text = f.read()

# Use a regular expression to split the text into references
references = re.split(' *\[[^\]]*\] *', text)

# Remove the first element of the list (it's empty because the text starts with a reference number)
references = references[1:]

# Open the .bib file in write mode
with open('my_references.bib', 'w') as f:
    # Loop over the references
    for ref in references:
        # Remove the trailing newline character
        ref = ref.rstrip('\n')

        try:
            # Fetch the BibTeX entry for this reference
            search_query = scholarly.search_pubs(ref)
            publication = next(search_query)
            bibtex = scholarly.bibtex(publication)

            # Write the BibTeX entry to the file, followed by a newline
            f.write(bibtex + '\n')
        except StopIteration:
            print(f"No results for reference: {ref}")


True
No results for reference: P. Angelov, Autonomous Learning Systems: From Data Streams to
Knowledge in Real-time.
New York: John Wiley & Sons, 2012.
No results for reference: E. Lughofer and M. Sayed-Mouchaweh, Predictive Maintenance in
Dynamic Systems — Advanced Methods, Decision Support Tools and
Real-World Applications.
New York: Springer, 2019.
No results for reference: E. Lughofer, “Evolving fuzzy and neuro-fuzzy systems: Fundamentals,
stability, explainability, useability, and applications,” in Handbook on
Computer Learning and Intelligence, P. Angelov, Ed. New York: World
Scientiﬁc, 2022, pp. 133–234.
No results for reference: X. Gu, J. Han, Q. Shen, and P. Angelov, “Autonomous learning
for fuzzy systems: a review,” Artiﬁcial Intelligence Review, vol.
https://doi.org/10.1007/s10462-022-10355-6, 2022.


The version of firefox cannot be detected. Trying with latest driver version


No results for reference: I. Khamassi, M. Sayed-Mouchaweh, M. Hammami, and K. Ghedira,
“Discussion and review on evolving data streams and concept drift
adapting,” Evolving Systems, vol. 9, no. 1, pp. 1–23, 2017.
No results for reference: P. Angelov, X. Gu, and J. Principe, “Autonomous learning multimodel
systems from data streams,” IEEE Transactions on Fuzzy Systems,
vol. 26, no. 4, pp. 2213–2224, 2018.


The version of firefox cannot be detected. Trying with latest driver version


No results for reference: E. Lughofer, “Single-pass active learning with conﬂict and ignorance,”
Evolving Systems, vol. 3, no. 4, pp. 251–271, 2012.
No results for reference: K. Subramanian, S. Suresh, and N. Sundararajan, “A metacognitive
neuro-fuzzy inference system (mcﬁs) for sequential classiﬁcation prob-
lems,” IEEE Transactions on Fuzzy Systems, vol. 21, no. 6, pp. 1080–
1095, 2013.
No results for reference: M. Pratama, S. Anavatti, and J. Lu, “Recurrent classiﬁer based on an
incremental meta-cognitive scaffolding algorithm,” IEEE Transactions
on Fuzzy Systems, vol. 23, no. 6, pp. 2048–2066, 2015.
No results for reference: D. Leite, G. Andonovski, I. Skrjanc, and F. Gomide, “Optimal rule-
based granular systems from data streams,” IEEE Transactions on Fuzzy
Systems, vol. 28, no. 3, pp. 583–596, 2020.
No results for reference: E. Lughofer and M. Pratama, “On-line active learning in data stream
regression using uncertainty sampling based on evolving generalized
fuzzy models,” IEEE 